# Advanced XPCS Correlation Analysis

This tutorial provides in-depth coverage of X-ray Photon Correlation Spectroscopy (XPCS) analysis techniques using the XPCS Toolkit. You'll learn advanced correlation function analysis, model fitting, and parameter extraction methods.

## Learning Objectives

By the end of this tutorial, you will:
1. Understand different correlation function models
2. Perform advanced fitting with multiple models
3. Extract physical parameters from correlation data
4. Analyze q-dependent dynamics
5. Assess data quality and statistical significance
6. Handle complex relaxation behaviors


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import stats
import pandas as pd
from pathlib import Path

# XPCS Toolkit imports
from xpcs_toolkit import XpcsDataFile
from xpcs_toolkit.module import g2mod

# Configure plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['lines.markersize'] = 4

print("Advanced XPCS correlation analysis tutorial")
print("XPCS Toolkit imported successfully!")

## Correlation Function Models

### 1. Single Exponential (Brownian Motion)

For simple diffusive systems:
$$g_2(\tau) = 1 + \beta \exp(-2\Gamma\tau)$$
where $\Gamma = Dq^2$ is the relaxation rate.

In [ ]:
def single_exponential(tau, beta, gamma, baseline=1.0):
    """
    Single exponential correlation function model.
    
    Parameters:
    -----------
    tau : array
        Delay times
    beta : float
        Coherence factor (0 < beta <= 1)
    gamma : float 
        Relaxation rate (1/s)
    baseline : float
        Long-time baseline (typically 1.0)
    
    Returns:
    --------
    g2 : array
        Correlation function values
    """
    return baseline + beta * np.exp(-2 * gamma * tau)

# Generate example data
tau_example = np.logspace(-5, -1, 50)
g2_single = single_exponential(tau_example, beta=0.8, gamma=5000)

plt.figure(figsize=(10, 6))
plt.semilogx(tau_example, g2_single, 'b-', linewidth=2, label='Single Exponential')
plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(τ)')
plt.title('Single Exponential Correlation Function')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("Single exponential model: g₂(τ) = 1 + β exp(-2Γτ)")
print("Characteristic features:")
print("- Simple exponential decay")
print("- Single characteristic time 1/(2Γ)")
print("- Typical for Brownian particles in dilute suspensions")

### 2. Stretched Exponential (KWW Function)

For systems with dynamic heterogeneity:
$$g_2(\tau) = 1 + \beta \exp(-2(\Gamma\tau)^\alpha)$$
where $\alpha$ (0 < α ≤ 1) is the stretching exponent.

In [ ]:
def stretched_exponential(tau, beta, gamma, alpha, baseline=1.0):
    """
    Stretched exponential (Kohlrausch-Williams-Watts) correlation function.
    
    Parameters:
    -----------
    alpha : float
        Stretching exponent (0 < alpha <= 1)
        alpha = 1: pure exponential
        alpha < 1: stretched (slower than exponential)
    """
    return baseline + beta * np.exp(-2 * (gamma * tau)**alpha)

# Compare different stretching exponents
alphas = [1.0, 0.8, 0.6, 0.4]
colors = ['blue', 'red', 'green', 'orange']

plt.figure(figsize=(12, 8))

for i, (alpha, color) in enumerate(zip(alphas, colors)):
    g2_stretched = stretched_exponential(tau_example, beta=0.8, gamma=5000, alpha=alpha)
    
    plt.subplot(2, 2, i+1)
    plt.semilogx(tau_example, g2_stretched, color=color, linewidth=2, 
                label=f'α = {alpha}')
    plt.xlabel('Delay time τ (s)')
    plt.ylabel('g₂(τ)')
    plt.title(f'Stretched Exponential (α = {alpha})')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    if alpha < 1.0:
        plt.text(0.02, 0.5, 'Slower decay\n(heterogeneous)', 
                transform=plt.gca().transAxes, fontsize=9,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print("Stretched exponential model: g₂(τ) = 1 + β exp(-2(Γτ)^α)")
print("Physical interpretation:")
print("- α = 1.0: Simple exponential (homogeneous dynamics)")
print("- α < 1.0: Stretched decay (heterogeneous dynamics)")
print("- Common in glasses, gels, and complex fluids")

### 3. Double Exponential

For systems with two distinct timescales:
$$g_2(\tau) = 1 + \beta_1 \exp(-2\Gamma_1\tau) + \beta_2 \exp(-2\Gamma_2\tau)$$

In [ ]:
def double_exponential(tau, beta1, gamma1, beta2, gamma2, baseline=1.0):
    """
    Double exponential correlation function.
    
    Parameters:
    -----------
    beta1, beta2 : float
        Amplitudes of fast and slow components
    gamma1, gamma2 : float
        Relaxation rates (gamma1 > gamma2 typically)
    """
    return baseline + beta1 * np.exp(-2 * gamma1 * tau) + beta2 * np.exp(-2 * gamma2 * tau)

# Example: fast and slow relaxation processes
g2_double = double_exponential(tau_example, 
                              beta1=0.6, gamma1=50000,  # Fast component
                              beta2=0.2, gamma2=1000)   # Slow component

# Compare with single exponentials
g2_fast_only = single_exponential(tau_example, beta=0.6, gamma=50000)
g2_slow_only = single_exponential(tau_example, beta=0.2, gamma=1000)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.semilogx(tau_example, g2_double, 'k-', linewidth=3, label='Double Exponential')
plt.semilogx(tau_example, g2_fast_only, 'r--', alpha=0.7, label='Fast Component Only')
plt.semilogx(tau_example, g2_slow_only, 'b--', alpha=0.7, label='Slow Component Only')
plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(τ)')
plt.title('Double Exponential Decomposition')
plt.legend()
plt.grid(True, alpha=0.3)

# Residual plot
plt.subplot(1, 2, 2)
g2_single_fit = single_exponential(tau_example, beta=0.8, gamma=10000)
residuals = g2_double - g2_single_fit

plt.semilogx(tau_example, residuals, 'ro-', markersize=3)
plt.axhline(0, color='k', linestyle='-', alpha=0.5)
plt.xlabel('Delay time τ (s)')
plt.ylabel('Residuals (Double - Single)')
plt.title('Evidence for Two Timescales')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Double exponential model: g₂(τ) = 1 + β₁exp(-2Γ₁τ) + β₂exp(-2Γ₂τ)")
print("Applications:")
print("- Binary mixtures with different particle sizes")
print("- Systems with fast local and slow collective motion")
print("- Polydisperse samples with broad size distributions")

## Advanced Fitting Procedures

### Model Comparison and Selection

In [ ]:
def fit_correlation_models(tau, g2, g2_err=None):
    """
    Fit multiple correlation function models and compare results.
    
    Returns dictionary with fit results and model comparison statistics.
    """
    results = {}
    
    # Prepare weights for fitting
    if g2_err is not None:
        weights = 1.0 / g2_err
        # Handle zero or very small errors
        weights = np.where(g2_err > 0, weights, 1.0)
    else:
        weights = None
    
    # 1. Single Exponential Fit
    try:
        p0_single = [0.8, 5000, 1.0]  # beta, gamma, baseline
        bounds_single = ([0, 0, 0.95], [1, np.inf, 1.05])
        
        popt_single, pcov_single = optimize.curve_fit(
            single_exponential, tau, g2, p0=p0_single, 
            sigma=g2_err, bounds=bounds_single, maxfev=5000
        )
        
        perr_single = np.sqrt(np.diag(pcov_single))
        g2_fit_single = single_exponential(tau, *popt_single)
        
        # Calculate chi-squared
        if g2_err is not None:
            chi2_single = np.sum(((g2 - g2_fit_single) / g2_err)**2)
        else:
            chi2_single = np.sum((g2 - g2_fit_single)**2)
        
        dof_single = len(g2) - len(popt_single)
        chi2_reduced_single = chi2_single / dof_single
        
        results['single'] = {
            'params': {'beta': popt_single[0], 'gamma': popt_single[1], 'baseline': popt_single[2]},
            'errors': {'beta': perr_single[0], 'gamma': perr_single[1], 'baseline': perr_single[2]},
            'fit_curve': g2_fit_single,
            'chi2': chi2_single,
            'chi2_reduced': chi2_reduced_single,
            'dof': dof_single,
            'n_params': len(popt_single)
        }
    except Exception as e:
        print(f"Single exponential fit failed: {e}")
        results['single'] = None
    
    # 2. Stretched Exponential Fit
    try:
        p0_stretched = [0.8, 5000, 0.8, 1.0]  # beta, gamma, alpha, baseline
        bounds_stretched = ([0, 0, 0.1, 0.95], [1, np.inf, 1.0, 1.05])
        
        popt_stretched, pcov_stretched = optimize.curve_fit(
            stretched_exponential, tau, g2, p0=p0_stretched,
            sigma=g2_err, bounds=bounds_stretched, maxfev=5000
        )
        
        perr_stretched = np.sqrt(np.diag(pcov_stretched))
        g2_fit_stretched = stretched_exponential(tau, *popt_stretched)
        
        if g2_err is not None:
            chi2_stretched = np.sum(((g2 - g2_fit_stretched) / g2_err)**2)
        else:
            chi2_stretched = np.sum((g2 - g2_fit_stretched)**2)
        
        dof_stretched = len(g2) - len(popt_stretched)
        chi2_reduced_stretched = chi2_stretched / dof_stretched
        
        results['stretched'] = {
            'params': {'beta': popt_stretched[0], 'gamma': popt_stretched[1], 
                      'alpha': popt_stretched[2], 'baseline': popt_stretched[3]},
            'errors': {'beta': perr_stretched[0], 'gamma': perr_stretched[1],
                      'alpha': perr_stretched[2], 'baseline': perr_stretched[3]},
            'fit_curve': g2_fit_stretched,
            'chi2': chi2_stretched,
            'chi2_reduced': chi2_reduced_stretched,
            'dof': dof_stretched,
            'n_params': len(popt_stretched)
        }
    except Exception as e:
        print(f"Stretched exponential fit failed: {e}")
        results['stretched'] = None
    
    return results

# Test with synthetic data
np.random.seed(42)  # For reproducible results

# Generate stretched exponential data with noise
tau_test = np.logspace(-5, -1, 40)
g2_true = stretched_exponential(tau_test, beta=0.75, gamma=8000, alpha=0.7)
noise_level = 0.01
g2_noisy = g2_true + noise_level * np.random.normal(0, 1, len(tau_test))
g2_errors = noise_level * np.ones(len(tau_test))

# Fit models
fit_results = fit_correlation_models(tau_test, g2_noisy, g2_errors)

# Plot comparison
plt.figure(figsize=(14, 10))

# Main plot with fits
plt.subplot(2, 2, 1)
plt.errorbar(tau_test, g2_noisy, yerr=g2_errors, fmt='ko', markersize=4, alpha=0.7, label='Data')
plt.semilogx(tau_test, g2_true, 'g-', linewidth=2, alpha=0.8, label='True (α=0.7)')

if fit_results['single'] is not None:
    plt.semilogx(tau_test, fit_results['single']['fit_curve'], 'r--', linewidth=2, 
                label=f"Single Exp (χ²ᵣ={fit_results['single']['chi2_reduced']:.2f})")

if fit_results['stretched'] is not None:
    plt.semilogx(tau_test, fit_results['stretched']['fit_curve'], 'b-', linewidth=2,
                label=f"Stretched (χ²ᵣ={fit_results['stretched']['chi2_reduced']:.2f})")

plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(τ)')
plt.title('Model Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Residuals plots
if fit_results['single'] is not None:
    plt.subplot(2, 2, 2)
    residuals_single = g2_noisy - fit_results['single']['fit_curve']
    plt.semilogx(tau_test, residuals_single, 'ro-', markersize=3, label='Single Exponential')
    plt.axhline(0, color='k', linestyle='-', alpha=0.5)
    plt.xlabel('Delay time τ (s)')
    plt.ylabel('Residuals')
    plt.title('Single Exponential Residuals')
    plt.grid(True, alpha=0.3)

if fit_results['stretched'] is not None:
    plt.subplot(2, 2, 3)
    residuals_stretched = g2_noisy - fit_results['stretched']['fit_curve']
    plt.semilogx(tau_test, residuals_stretched, 'bo-', markersize=3, label='Stretched Exponential')
    plt.axhline(0, color='k', linestyle='-', alpha=0.5)
    plt.xlabel('Delay time τ (s)')
    plt.ylabel('Residuals')
    plt.title('Stretched Exponential Residuals')
    plt.grid(True, alpha=0.3)

# Parameter comparison table
plt.subplot(2, 2, 4)
plt.axis('off')

if fit_results['single'] is not None and fit_results['stretched'] is not None:
    table_data = [
        ['Parameter', 'True', 'Single Exp', 'Stretched'],
        ['β', '0.75', f"{fit_results['single']['params']['beta']:.3f}±{fit_results['single']['errors']['beta']:.3f}", 
         f"{fit_results['stretched']['params']['beta']:.3f}±{fit_results['stretched']['errors']['beta']:.3f}"],
        ['Γ (s⁻¹)', '8000', f"{fit_results['single']['params']['gamma']:.0f}±{fit_results['single']['errors']['gamma']:.0f}",
         f"{fit_results['stretched']['params']['gamma']:.0f}±{fit_results['stretched']['errors']['gamma']:.0f}"],
        ['α', '0.70', '1.0 (fixed)', f"{fit_results['stretched']['params']['alpha']:.3f}±{fit_results['stretched']['errors']['alpha']:.3f}"],
        ['χ²ᵣ', '-', f"{fit_results['single']['chi2_reduced']:.2f}", f"{fit_results['stretched']['chi2_reduced']:.2f}"]
    ]
    
    table = plt.table(cellText=table_data[1:], colLabels=table_data[0],
                     cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)

plt.tight_layout()
plt.show()

# Print model comparison summary
print("\n" + "="*50)
print("MODEL COMPARISON SUMMARY")
print("="*50)

if fit_results['single'] is not None:
    print(f"\nSingle Exponential:")
    print(f"  χ²ᵣ = {fit_results['single']['chi2_reduced']:.3f}")
    print(f"  β = {fit_results['single']['params']['beta']:.3f} ± {fit_results['single']['errors']['beta']:.3f}")
    print(f"  Γ = {fit_results['single']['params']['gamma']:.1f} ± {fit_results['single']['errors']['gamma']:.1f} s⁻¹")

if fit_results['stretched'] is not None:
    print(f"\nStretched Exponential:")
    print(f"  χ²ᵣ = {fit_results['stretched']['chi2_reduced']:.3f}")
    print(f"  β = {fit_results['stretched']['params']['beta']:.3f} ± {fit_results['stretched']['errors']['beta']:.3f}")
    print(f"  Γ = {fit_results['stretched']['params']['gamma']:.1f} ± {fit_results['stretched']['errors']['gamma']:.1f} s⁻¹")
    print(f"  α = {fit_results['stretched']['params']['alpha']:.3f} ± {fit_results['stretched']['errors']['alpha']:.3f}")

# Model selection criterion
if fit_results['single'] is not None and fit_results['stretched'] is not None:
    if fit_results['stretched']['chi2_reduced'] < fit_results['single']['chi2_reduced']:
        print(f"\n✓ Stretched exponential model provides better fit (lower χ²ᵣ)")
        if fit_results['stretched']['params']['alpha'] < 0.95:
            print(f"  α = {fit_results['stretched']['params']['alpha']:.3f} indicates significant stretching")
    else:
        print(f"\n✓ Single exponential model is adequate")

## Q-dependent Analysis

### Extracting Diffusion Coefficients

In [ ]:
def analyze_q_dependence(q_values, tau_data, g2_data, g2_errors=None):
    """
    Analyze q-dependence of correlation functions to extract diffusion coefficients.
    
    Returns:
    --------
    results : dict
        Dictionary containing relaxation rates, diffusion coefficients, and fits
    """
    n_q = len(q_values)
    
    results = {
        'q_values': q_values,
        'gamma': np.zeros(n_q),
        'gamma_err': np.zeros(n_q),
        'beta': np.zeros(n_q),
        'beta_err': np.zeros(n_q),
        'chi2_reduced': np.zeros(n_q),
        'fits': []
    }
    
    for i, q in enumerate(q_values):
        g2_q = g2_data[:, i]
        g2_err_q = g2_errors[:, i] if g2_errors is not None else None
        
        # Fit single exponential for each q
        try:
            p0 = [0.8, 5000, 1.0]  # beta, gamma, baseline
            bounds = ([0, 0, 0.95], [1, np.inf, 1.05])
            
            popt, pcov = optimize.curve_fit(
                single_exponential, tau_data, g2_q,
                p0=p0, sigma=g2_err_q, bounds=bounds, maxfev=5000
            )
            
            perr = np.sqrt(np.diag(pcov))
            g2_fit = single_exponential(tau_data, *popt)
            
            # Calculate chi-squared
            if g2_err_q is not None:
                chi2 = np.sum(((g2_q - g2_fit) / g2_err_q)**2)
            else:
                chi2 = np.sum((g2_q - g2_fit)**2)
            
            dof = len(g2_q) - len(popt)
            chi2_reduced = chi2 / dof
            
            results['beta'][i] = popt[0]
            results['beta_err'][i] = perr[0]
            results['gamma'][i] = popt[1]
            results['gamma_err'][i] = perr[1]
            results['chi2_reduced'][i] = chi2_reduced
            results['fits'].append(g2_fit)
            
        except Exception as e:
            print(f"Fit failed for q = {q:.3f} Å⁻¹: {e}")
            results['beta'][i] = np.nan
            results['beta_err'][i] = np.nan
            results['gamma'][i] = np.nan
            results['gamma_err'][i] = np.nan
            results['chi2_reduced'][i] = np.nan
            results['fits'].append(None)
    
    # Calculate diffusion coefficients: D = Γ/q²
    valid_mask = ~np.isnan(results['gamma'])
    if np.any(valid_mask):
        q_si = q_values * 1e10  # Convert Å⁻¹ to m⁻¹
        
        results['D'] = results['gamma'] / (q_si**2)
        results['D_err'] = results['gamma_err'] / (q_si**2)
        
        # Fit D vs q² to check for q-independence
        q2_valid = (q_si[valid_mask])**2
        D_valid = results['D'][valid_mask]
        D_err_valid = results['D_err'][valid_mask]
        
        # Weighted average of D
        weights = 1.0 / (D_err_valid**2)
        D_avg = np.average(D_valid, weights=weights)
        D_avg_err = np.sqrt(1.0 / np.sum(weights))
        
        results['D_average'] = D_avg
        results['D_average_err'] = D_avg_err
        
        # Linear fit to assess q-independence
        try:
            slope, intercept, r_value, p_value, std_err = stats.linregress(q2_valid, D_valid)
            results['D_vs_q2_slope'] = slope
            results['D_vs_q2_slope_err'] = std_err
            results['D_vs_q2_intercept'] = intercept
            results['D_vs_q2_r_squared'] = r_value**2
            results['D_vs_q2_p_value'] = p_value
        except:
            pass
    
    return results

# Generate multi-q synthetic data
np.random.seed(123)
q_range = np.linspace(0.02, 0.1, 8)
tau_range = np.logspace(-5, -1, 35)
D_true = 1.5e-12  # m²/s

# Generate correlation functions for different q values
g2_multi = np.zeros((len(tau_range), len(q_range)))
g2_err_multi = np.zeros((len(tau_range), len(q_range)))

for i, q in enumerate(q_range):
    q_si = q * 1e10  # m⁻¹
    gamma_true = D_true * q_si**2
    
    g2_clean = single_exponential(tau_range, beta=0.8, gamma=gamma_true)
    noise = 0.008 * np.random.normal(0, 1, len(tau_range))
    g2_multi[:, i] = g2_clean + noise
    g2_err_multi[:, i] = 0.008 * np.ones(len(tau_range))

# Analyze q-dependence
q_analysis = analyze_q_dependence(q_range, tau_range, g2_multi, g2_err_multi)

# Plot results
fig = plt.figure(figsize=(16, 12))

# 1. Correlation functions for all q-values
plt.subplot(2, 3, 1)
colors = plt.cm.viridis(np.linspace(0, 1, len(q_range)))

for i, (q, color) in enumerate(zip(q_range, colors)):
    plt.errorbar(tau_range[::3], g2_multi[::3, i], yerr=g2_err_multi[::3, i], 
                 color=color, fmt='o', markersize=3, alpha=0.7, label=f'q={q:.3f}')
    if q_analysis['fits'][i] is not None:
        plt.semilogx(tau_range, q_analysis['fits'][i], color=color, linewidth=1.5)

plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(q,τ)')
plt.title('Multi-q Correlation Functions')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.grid(True, alpha=0.3)

# 2. Relaxation rate vs q²
plt.subplot(2, 3, 2)
valid_mask = ~np.isnan(q_analysis['gamma'])
q_squared = (q_range * 1e10)**2

if np.any(valid_mask):
    plt.errorbar(q_squared[valid_mask], q_analysis['gamma'][valid_mask], 
                 yerr=q_analysis['gamma_err'][valid_mask],
                 fmt='bo', markersize=6, capsize=3, label='Data')
    
    # Theoretical line: Γ = D*q²
    gamma_theory = D_true * q_squared
    plt.plot(q_squared, gamma_theory, 'r-', linewidth=2, alpha=0.7, 
             label=f'Theory: D={D_true:.2e} m²/s')

plt.xlabel('q² (m⁻²)')
plt.ylabel('Relaxation rate Γ (s⁻¹)')
plt.title('Γ vs q² (Linear Dependence Expected)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ticklabel_format(style='scientific', axis='both', scilimits=(0,0))

# 3. Diffusion coefficient vs q
plt.subplot(2, 3, 3)
if 'D' in q_analysis and np.any(valid_mask):
    plt.errorbar(q_range[valid_mask], q_analysis['D'][valid_mask], 
                 yerr=q_analysis['D_err'][valid_mask],
                 fmt='go', markersize=6, capsize=3, label='Extracted D')
    
    # Average line
    if 'D_average' in q_analysis:
        plt.axhline(q_analysis['D_average'], color='blue', linestyle='--', alpha=0.7,
                   label=f"Average: {q_analysis['D_average']:.2e} ± {q_analysis['D_average_err']:.2e}")
    
    # True value
    plt.axhline(D_true, color='red', linestyle='-', alpha=0.7,
               label=f'True: {D_true:.2e} m²/s')

plt.xlabel('q (Å⁻¹)')
plt.ylabel('Diffusion Coefficient (m²/s)')
plt.title('D vs q (Should be q-independent)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# 4. Coherence factor vs q
plt.subplot(2, 3, 4)
if np.any(valid_mask):
    plt.errorbar(q_range[valid_mask], q_analysis['beta'][valid_mask], 
                 yerr=q_analysis['beta_err'][valid_mask],
                 fmt='mo', markersize=6, capsize=3)

plt.axhline(0.8, color='red', linestyle='--', alpha=0.7, label='True β = 0.8')
plt.xlabel('q (Å⁻¹)')
plt.ylabel('Coherence factor β')
plt.title('β vs q')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0.6, 1.0)

# 5. Fit quality (chi-squared)
plt.subplot(2, 3, 5)
if np.any(valid_mask):
    plt.plot(q_range[valid_mask], q_analysis['chi2_reduced'][valid_mask], 'ko-', markersize=6)

plt.axhline(1.0, color='red', linestyle='--', alpha=0.7, label='χ²ᵣ = 1 (ideal)')
plt.xlabel('q (Å⁻¹)')
plt.ylabel('Reduced χ²')
plt.title('Fit Quality vs q')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Summary statistics table
plt.subplot(2, 3, 6)
plt.axis('off')

if 'D_average' in q_analysis:
    # Calculate particle size using Stokes-Einstein
    kT = 4.1e-21  # J (room temperature)
    eta = 1e-3    # Pa·s (water viscosity)
    R_h = kT / (6 * np.pi * eta * q_analysis['D_average'])
    R_h_err = kT / (6 * np.pi * eta) * q_analysis['D_average_err'] / (q_analysis['D_average']**2)
    
    summary_text = f"""
ANALYSIS SUMMARY
{'='*30}

Diffusion Coefficient:
  D = ({q_analysis['D_average']:.2e} ± {q_analysis['D_average_err']:.2e}) m²/s
  True value: {D_true:.2e} m²/s
  Relative error: {abs(q_analysis['D_average']-D_true)/D_true*100:.1f}%

Hydrodynamic Radius:
  Rₕ = ({R_h*1e9:.1f} ± {R_h_err*1e9:.1f}) nm

Q-dependence Check:
  D should be q-independent for
  simple Brownian motion
  
Fit Quality:
  Average χ²ᵣ = {np.nanmean(q_analysis['chi2_reduced']):.2f}
  Good fits: χ²ᵣ ≈ 1
"""
    
    plt.text(0.05, 0.95, summary_text, transform=plt.gca().transAxes, 
             fontsize=9, verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Q-DEPENDENT ANALYSIS RESULTS")
print("="*60)

if 'D_average' in q_analysis:
    print(f"\nExtracted Diffusion Coefficient:")
    print(f"  D = ({q_analysis['D_average']:.2e} ± {q_analysis['D_average_err']:.2e}) m²/s")
    print(f"  True value: {D_true:.2e} m²/s")
    print(f"  Accuracy: {abs(q_analysis['D_average']-D_true)/D_true*100:.1f}% error")
    
    print(f"\nParticle Size (Stokes-Einstein):")
    print(f"  Hydrodynamic radius: {R_h*1e9:.1f} ± {R_h_err*1e9:.1f} nm")
    
    if 'D_vs_q2_slope' in q_analysis:
        print(f"\nQ-independence Check:")
        print(f"  D vs q² slope: {q_analysis['D_vs_q2_slope']:.2e} ± {q_analysis['D_vs_q2_slope_err']:.2e}")
        print(f"  Expected slope: ~0 for Brownian motion")
        if abs(q_analysis['D_vs_q2_slope']) < 3*q_analysis['D_vs_q2_slope_err']:
            print(f"  ✓ Slope consistent with zero (Brownian motion confirmed)")
        else:
            print(f"  ⚠ Significant q-dependence detected (non-Brownian behavior)")

## Advanced Topics

### Non-ergodic Systems

In [ ]:
# Example of non-ergodic behavior (g2 doesn't reach 1 at long times)
def non_ergodic_correlation(tau, beta_erg, gamma, beta_non_erg, baseline=1.0):
    """
    Non-ergodic correlation function with persistent correlations.
    
    g2(τ) = baseline + β_non_erg + β_erg * exp(-2Γτ)
    
    For ergodic systems: β_non_erg = 0
    For non-ergodic systems: β_non_erg > 0
    """
    return baseline + beta_non_erg + beta_erg * np.exp(-2 * gamma * tau)

# Generate examples of ergodic vs non-ergodic behavior
tau_demo = np.logspace(-5, 1, 60)

g2_ergodic = non_ergodic_correlation(tau_demo, beta_erg=0.8, gamma=1000, beta_non_erg=0.0)
g2_non_ergodic = non_ergodic_correlation(tau_demo, beta_erg=0.6, gamma=1000, beta_non_erg=0.3)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.semilogx(tau_demo, g2_ergodic, 'b-', linewidth=2, label='Ergodic (liquid)')
plt.semilogx(tau_demo, g2_non_ergodic, 'r-', linewidth=2, label='Non-ergodic (glass/gel)')
plt.axhline(1.0, color='k', linestyle='--', alpha=0.5, label='g₂ = 1 (ergodic limit)')
plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(τ)')
plt.title('Ergodic vs Non-ergodic Behavior')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0.9, 2.1)

# Highlight long-time behavior
plt.subplot(1, 2, 2)
plt.semilogx(tau_demo[30:], g2_ergodic[30:], 'b-', linewidth=2, label='Ergodic')
plt.semilogx(tau_demo[30:], g2_non_ergodic[30:], 'r-', linewidth=2, label='Non-ergodic')
plt.axhline(1.0, color='k', linestyle='--', alpha=0.5)
plt.axhline(1.3, color='r', linestyle=':', alpha=0.7, label='Non-ergodic plateau')
plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(τ→∞)')
plt.title('Long-time Behavior')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0.95, 1.35)

plt.tight_layout()
plt.show()

print("Non-ergodic Systems:")
print("- g₂(τ→∞) > 1 indicates frozen configurations")
print("- Common in glasses, gels, and jammed systems")
print("- Non-ergodic parameter: f = β_non_erg / (β_erg + β_non_erg)")
print("- f = 0: fully ergodic, f = 1: completely frozen")

### Two-time Correlation Analysis

In [ ]:
# Example of aging behavior in two-time correlation functions
def aging_correlation(tau, t_wait, beta, gamma_0, aging_exponent):
    """
    Two-time correlation function showing aging behavior.
    
    For aging systems, the relaxation rate depends on waiting time:
    γ(t_wait) = γ₀ * (t_wait)^(-aging_exponent)
    """
    gamma_eff = gamma_0 * (t_wait ** (-aging_exponent))
    return 1.0 + beta * np.exp(-2 * gamma_eff * tau)

# Generate aging correlation functions
tau_aging = np.logspace(-3, 2, 50)
wait_times = [1, 10, 100, 1000]  # seconds
colors_aging = ['blue', 'green', 'orange', 'red']

plt.figure(figsize=(10, 6))

for t_wait, color in zip(wait_times, colors_aging):
    g2_aging = aging_correlation(tau_aging, t_wait, beta=0.8, gamma_0=1000, aging_exponent=0.3)
    plt.semilogx(tau_aging, g2_aging, color=color, linewidth=2, 
                label=f't_wait = {t_wait} s')

plt.xlabel('Delay time τ (s)')
plt.ylabel('g₂(τ, t_wait)')
plt.title('Aging Behavior in Two-time Correlations')
plt.legend()
plt.grid(True, alpha=0.3)

# Add annotation
plt.annotate('Slower relaxation\nfor longer wait times', 
            xy=(10, 1.3), xytext=(100, 1.6),
            arrowprops=dict(arrowstyle='->', color='black', alpha=0.7),
            fontsize=10, ha='center')

plt.show()

print("Aging in XPCS:")
print("- Two-time correlation: g₂(t₁, t₂) depends on both t₁ and t₂")
print("- In equilibrium: g₂(t₁, t₂) = g₂(|t₂-t₁|) (time-translation invariant)")
print("- Aging systems: g₂ depends on waiting time t_wait")
print("- Examples: glasses, gels, colloidal suspensions near jamming")

## Summary and Best Practices

### Key Takeaways

1. **Model Selection**: Always compare multiple models and use statistical criteria (χ²ᵣ, AIC, residual analysis)

2. **Q-dependence**: Verify that extracted diffusion coefficients are q-independent for Brownian motion

3. **Data Quality**: Check for proper baseline behavior, statistical errors, and systematic effects

4. **Physical Interpretation**: Connect fitting parameters to physical quantities (D, R_h, relaxation mechanisms)

5. **Advanced Phenomena**: Be aware of non-ergodic behavior, aging, and other complex dynamics

### Recommended Analysis Workflow

```python
# 1. Load and inspect data
xf = XpcsDataFile('your_data.h5')
q, tau, g2, g2_err, labels = xf.get_g2_data()

# 2. Visual inspection
g2mod.pg_plot(hdl, [xf], q_range, t_range, y_range)

# 3. Model fitting and comparison
results = fit_correlation_models(tau, g2, g2_err)

# 4. Q-dependent analysis
q_results = analyze_q_dependence(q, tau, g2, g2_err)

# 5. Physical interpretation
# Extract D, calculate R_h, assess dynamics type
```

### Next Steps

- **03_saxs_structure_analysis.ipynb**: SAXS analysis techniques
- **04_stability_assessment.ipynb**: Time-resolved analysis  
- **05_advanced_fitting.ipynb**: Custom models and global fitting
